Zoning program

In [1]:
from skimage import io
from skimage.transform import resize
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
import numpy as np
import math


def imgToBinaryMatrix(image):
    image = resize(image, (20, 20), preserve_range=True).astype('uint8')
    threshold = 150
    # make all pixels < threshold black
    binarized = 1.0 * (image > threshold)

    # convert 3d array to 2d array
    return binarized[:, :, 0]


def convertMatrixTo3x3(matrix):
    xs = matrix.shape[0]//3  # division lines for the picture
    ys = matrix.shape[1]//3

    # now slice up the image (in a shape that works well with subplots)
    newMatrix = [[matrix[0:xs, 0:ys], matrix[0:xs, ys:ys*2], matrix[0:xs, ys*2:ys*3]],
                 [matrix[xs:xs*2, 0:ys], matrix[xs:xs*2, ys:ys*2],
                     matrix[xs:xs*2, ys*2:ys*3]],
                 [matrix[xs*2:xs*3, 0:ys], matrix[xs*2:xs*3, ys:ys*2],
                     matrix[xs*2:xs*3, ys*2:ys*3]],
                 ]
    return newMatrix


def convertMatrixTo4x4(matrix):
    xs = matrix.shape[0]//4  # division lines for the picture
    ys = matrix.shape[1]//4

    # now slice up the image (in a shape that works well with subplots)
    newMatrix = [[matrix[0:xs, 0:ys], matrix[0:xs, ys:ys*2], matrix[0:xs, ys*2:ys*3], matrix[0:xs, ys*3:ys*4]],
                 [matrix[xs:xs*2, 0:ys], matrix[xs:xs*2, ys:ys*2],
                     matrix[xs:xs*2, ys*2:ys*3], matrix[xs:xs, ys*3:ys*4]],
                 [matrix[xs*2:xs*3, 0:ys], matrix[xs*2:xs*3, ys:ys*2],
                     matrix[xs*2:xs*3, ys*2:ys*3], matrix[xs*2:xs*3, ys*3:ys*4]],
                 [matrix[xs*3:xs*4, 0:ys], matrix[xs*3:xs*4, ys:ys*2],
                  matrix[xs*3:xs*4, ys*3:ys*4], matrix[xs*3:xs*4, ys*3:ys*4]]
                 ]
    return newMatrix


def convert3x3ToVector(matrix):
    vector = []
    for i in range(3):
        for j in range(3):
            vector.append(np.sum(matrix[i][j]))
    return vector


def convert4x4ToVector(matrix):
    vector = []
    for i in range(4):
        for j in range(4):
            vector.append(np.sum(matrix[i][j]))
    return vector


def euclideanDistance(v1, v2):
    distance = 0
    for i in range(len(v1)):
        distance += math.pow((v1[i] - v2[i]), 2)
    return math.sqrt(distance)


def getComparedVectorsExcept(number, nth):
    vectors = []
    for i in range(10):
        for j in range(1, 11):
            if i == number and j == nth:
                continue
            name = f'baseProjetOCR/{i}_{j}.png'
            matrixImg = imgToBinaryMatrix(io.imread(name))
            matrixImg = convertMatrixTo4x4(matrixImg)
            vectors.append((name, convert4x4ToVector(matrixImg)))
    return vectors


def getFileNameCorrespondingTo(img, comparedVectors):
    matrixImg = imgToBinaryMatrix(img)
    meanKernel = np.full((3, 3), 1.0/9)
    matrixImg = ndi.correlate(matrixImg, meanKernel)
    matrixImg = convertMatrixTo4x4(matrixImg)
    sourceVector = convert4x4ToVector(matrixImg)

    allDistancesZoning = [euclideanDistance(sourceVector, comparedVectors[i][1]) for i in range(len(comparedVectors))]

    indexOfMinDistance = allDistancesZoning.index(min(allDistancesZoning))
    return comparedVectors[indexOfMinDistance][0]


def getNumberOfFileName(s):
    return int(s.split("/", 1)[1][0])


def confusionMatrix():
    mat = np.zeros(shape=(10, 10))
    for i in range(10):
        for j in range(11):
            if j == 0:
                continue
            vectors = getComparedVectorsExcept(i, j)
            n = getNumberOfFileName(getFileNameCorrespondingTo(
                io.imread(f'baseProjetOCR/{i}_{j}.png'), vectors))
            mat[i][n] += 1
    return mat


zoningConfMatrix = confusionMatrix()
print("Percentage : " + str(np.trace(zoningConfMatrix)))
print("Confusion matrix :\n" + str(zoningConfMatrix))


/home/gaabin/.local/lib/python3.8/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


Percentage : 88.0
Confusion matrix :
[[10.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  8.  0.  0.  1.  0.  0.  0.  0.  1.]
 [ 0.  0. 10.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  8.  0.  0.  0.  1.  0.  1.]
 [ 0.  0.  0.  0.  9.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. 10.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. 10.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  9.  0.  0.]
 [ 1.  0.  1.  1.  0.  0.  0.  0.  7.  0.]
 [ 0.  0.  0.  2.  0.  0.  0.  0.  1.  7.]]


Confusion matrix by using 3 algorithms

In [2]:

def bestConfMatrix(zoningMatrix, profilMatrix, knnMatrix):
    mat = np.zeros(shape=(10, 10))
    for i in range(10):
        if zoningMatrix[i][i] > profilMatrix[i][i] and zoningMatrix[i][i] > knnMatrix[i][i]:
            mat[i] = zoningMatrix[i]
        elif profilMatrix[i][i] > knnMatrix[i][i]:
            mat[i] = profilMatrix[i]
        else:
            mat[i] = knnMatrix[i]
    return mat


array1 = np.array([[8, 0, 1, 1, 0, 0, 0, 0, 0, 0],
                   [0, 10, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 10, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 9, 0, 0, 0, 1, 0, 0],
                   [0, 0, 0, 0, 10, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 10, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 9, 1, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 10, 0, 0],
                   [0, 0, 2, 0, 0, 0, 0, 0, 7, 1],
                   [0, 0, 0, 2, 0, 1, 0, 0, 1, 6]])
array2 = np.array([[10, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 8, 0, 0, 1, 0, 0, 0, 1, 0],
                   [0, 0, 10, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 8, 0, 0, 0, 1, 0, 1],
                   [0, 0, 0, 0, 10, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 10, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 10, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 10, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 9, 1],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0, 10]])
test = bestConfMatrix(zoningConfMatrix, array1, array2)
print("Percentage : " + str(np.trace(test)))
print("Confusion matrix :\n" + str(test))


Percentage : 98.0
Confusion matrix :
[[10.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. 10.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. 10.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  9.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0. 10.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. 10.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. 10.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. 10.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  9.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0. 10.]]
